In [ ]:
import pandas as pd
import numpy as np
from transformers import AutoTokenizer

In [ ]:
df = pd.read_csv('data/processed/merged_dataset.csv')
df.head()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
sentences = df['open_response'].tolist()
encoded = tokenizer(sentences, truncation=False)

seq_lengths = [np.count_nonzero(x) for x in encoded['input_ids']]
idx_long_seq = [i for i, x in enumerate(seq_lengths) if x > 512]
print(f'{len(idx_long_seq)} sentences longer than 512 tokens.')
print(f'Lengths: {sorted([seq_lengths[i] for i in idx_long_seq])}')
# for i in idx_long_seq[:1]:
#     print(tokenizer.decode(encoded['input_ids'][i]))

Given the existence of sentences longer than 512, we must truncate or split in some way. As it is a small number of records, we choose to truncate.

## Prepare in huggingface.datasets

In [ ]:
from sklearn.model_selection import train_test_split
# from numpy.random import default_rng
# rng = default_rng(87)

documents = df['open_response'].tolist()
label2id = {
    'No Schooling': 0,
    'Primary School': 1,
    'High School': 2,
    'College or Higher': 3
}
labels = df['g5_06a'].map(label2id).tolist()

# convert 
# create train/test split
train_texts, test_texts, train_labels, test_labels = train_test_split(documents, labels, test_size=.15, random_state=8573)

# create train/validation split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_texts, train_labels, test_size=.15, random_state=3820)


# tokenize 
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_texts, truncation=True, padding=True)


# create a class for the dataset
import torch

class VerbalAutopsyDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = VerbalAutopsyDataset(train_encodings, train_labels)
val_dataset = VerbalAutopsyDataset(val_encodings, val_labels)
test_dataset = VerbalAutopsyDataset(test_encodings, test_labels)


Fine tune the BERT model using the created dataset.

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()